
# 품목 통합 파이프라인 (배치 버전)

- 경로: `C:\Users\User\Desktop\GDF_Final_G3\GDF_Final_G3\도메인전처리\raw`  
- 입력 예시 및 필터:
  - **마늘.csv** → `['깐마늘','깐마늘 남도','깐마늘 대서']`
  - **청고추.csv** → `['풋고추(일반)','롱그린']`
  - **파프리카.csv** → `['녹색파프리카','청색파프리카']`
- 출력: 각 파일별로 `*_머지.csv`, `*(통합)_최종_데이터.csv` 저장

> 아래 셀을 위에서부터 순서대로 실행하세요. 파일/필터를 바꾸고 싶으면 **설정 셀**만 수정하면 됩니다.


In [4]:
# -*- coding: utf-8 -*-
# 한 셀 실행: 파일별 필터 → 통합(가중평균) → 전년/평년/증감률 → 저장
import os
import numpy as np
import pandas as pd

# ===== 경로/파일별 필터 설정 =====
BASE_DIR = r"C:/Users/User/Desktop/GDF_Final_G3/GDF_Final_G3/도메인전처리/이어붙이기data/"
ITEM_FILTERS = {
    "마늘.csv":    ["깐마늘","깐마늘 남도","깐마늘 대서"],
    "청고추.csv":  ["풋고추(일반)","롱그린"],
    "파프리카.csv":["녹색파프리카","청색파프리카"],
}
FINAL_SUFFIX  = "(통합)_최종_데이터.csv"

# ===== 유틸 =====
def read_csv_smart(path: str) -> pd.DataFrame:
    for enc in ("cp949", "utf-8-sig"):
        try:
            return pd.read_csv(path, encoding=enc)
        except Exception:
            pass
    raise RuntimeError(f"CSV 로드 실패(인코딩 확인): {path}")

def to_numeric_clean(s: pd.Series) -> pd.Series:
    if pd.api.types.is_numeric_dtype(s):
        return s
    return pd.to_numeric(
        s.astype(str)
         .str.replace(",", "", regex=False)
         .str.replace(" ", "", regex=False)
         .str.replace("%", "", regex=False)
         .str.replace("\u2212", "-", regex=False),
        errors="coerce"
    )

def safe_percentage(curr: pd.Series, prev: pd.Series) -> pd.Series:
    mask = (pd.notna(curr)) & (pd.notna(prev)) & (prev != 0)
    out = pd.Series(np.nan, index=curr.index)
    out[mask] = ((curr[mask] - prev[mask]) / prev[mask]) * 100
    return out

def calculate_comparison_data(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out["연도"] = out["거래일자"].dt.year
    out["월"]  = out["거래일자"].dt.month
    out["일"]  = out["거래일자"].dt.day

    prev = out[["연도","월","일","반입량","금액"]].copy()
    prev["연도"] = prev["연도"] + 1
    prev = prev.rename(columns={"반입량":"전년 반입량","금액":"전년 금액"})
    out = out.merge(prev, on=["연도","월","일"], how="left")

    monthly = out.groupby("월")[["반입량","금액"]].mean().rename(
        columns={"반입량":"평년 반입량","금액":"평년 금액"}
    )
    out = out.merge(monthly, on="월", how="left")

    out["전년 반입량 증감률(%)"] = safe_percentage(out["반입량"], out["전년 반입량"])
    out["전년 금액 증감률(%)"]   = safe_percentage(out["금액"], out["전년 금액"])
    out["평년 반입량 증감률(%)"] = safe_percentage(out["반입량"], out["평년 반입량"])
    out["평년 금액 증감률(%)"]   = safe_percentage(out["금액"], out["평년 금액"])

    return out.drop(columns=["연도","월","일"])

def integrate_items_extended(df: pd.DataFrame, keep_items: list[str], label: str) -> pd.DataFrame:
    if "거래일자" not in df.columns:
        raise ValueError("필수 컬럼 '거래일자'가 없습니다.")
    df = df.copy()
    df["거래일자"] = pd.to_datetime(df["거래일자"], errors="coerce")
    df = df.dropna(subset=["거래일자"])

    filtered = df[df.get("품목명", "").isin(keep_items)].copy()
    all_cols = [
        "거래일자","품목명","반입량","금액",
        "전년 반입량","전년 반입량 증감률(%)",
        "전년 금액","전년 금액 증감률(%)",
        "평년 반입량","평년 반입량 증감률(%)",
        "평년 금액","평년 금액 증감률(%)",
    ]
    available = [c for c in all_cols if c in filtered.columns]
    wdf = filtered[available].copy()

    num_cols = [
        "반입량","금액",
        "전년 반입량","전년 반입량 증감률(%)",
        "전년 금액","전년 금액 증감률(%)",
        "평년 반입량","평년 반입량 증감률(%)",
        "평년 금액","평년 금액 증감률(%)",
    ]
    for c in num_cols:
        if c in wdf.columns:
            wdf[c] = to_numeric_clean(wdf[c])

    rows = []
    for date, g in wdf.groupby("거래일자"):
        valid = g[(g.get("반입량").notna()) & (g["반입량"] > 0)]
        if valid.empty:
            continue
        total_qty = valid["반입량"].sum()
        row = {"거래일자": date, "품목명": label, "반입량": total_qty}
        for c in num_cols:
            if c == "반입량" or c not in valid.columns:
                continue
            vv = valid[valid[c].notna()]
            if vv.empty:
                row[c] = np.nan
            else:
                w = vv["반입량"] / vv["반입량"].sum()
                row[c] = (vv[c] * w).sum()
        rows.append(row)

    if not rows:
        return pd.DataFrame()

    res = pd.DataFrame(rows).sort_values("거래일자").reset_index(drop=True)

    need_prev = ("전년 반입량" not in res.columns) or res["전년 반입량"].isna().all()
    need_avg  = ("평년 반입량" not in res.columns) or res["평년 반입량"].isna().all()
    if need_prev or need_avg:
        res = calculate_comparison_data(res)

    return res

# ===== 배치 실행 =====
os.makedirs(BASE_DIR, exist_ok=True)
for fname, keep_list in ITEM_FILTERS.items():
    in_path = os.path.join(BASE_DIR, fname)
    stem = os.path.splitext(fname)[0]
    final_path  = os.path.join("./raw/drop", f"{stem}{FINAL_SUFFIX}")

    df = read_csv_smart(in_path)
    req = {"거래일자","품목명","반입량","금액"}
    miss = req - set(df.columns)
    if miss:
        raise ValueError(f"필수 컬럼 누락({fname}): {sorted(miss)}")

    # 필터 저장
    df_filtered = df[df["품목명"].isin(keep_list)].copy()
    df_filtered.to_csv(merged_path, index=False, encoding="utf-8-sig")

    # 통합 (품목명은 CSV 이름으로 고정)
    integrated = integrate_items_extended(df_filtered, keep_list, stem)

    if integrated.empty:
        print(f"[SKIP] 통합 결과 없음: {fname}")
        continue

    final_cols = [
        "거래일자","품목명","반입량","금액",
        "전년 반입량","전년 반입량 증감률(%)",
        "전년 금액","전년 금액 증감률(%)",
        "평년 반입량","평년 반입량 증감률(%)",
        "평년 금액","평년 금액 증감률(%)",
    ]
    final_cols = [c for c in final_cols if c in integrated.columns]
    integrated = integrated[final_cols].sort_values("거래일자").reset_index(drop=True)
    integrated.to_csv(final_path, index=False, encoding="utf-8-sig")
    print(f"[DONE] {fname} → {os.path.basename(final_path)}")


[DONE] 마늘.csv → 마늘(통합)_최종_데이터.csv
[DONE] 청고추.csv → 청고추(통합)_최종_데이터.csv
[DONE] 파프리카.csv → 파프리카(통합)_최종_데이터.csv
